## About

Detect dogs or cats in images.

Based on Jeremy Howard's fast.ai [lesson 1](https://github.com/fastai/fastai/blob/master/courses/dl1/lesson1.ipynb).

## Setup

In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Import libraries
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [ ]:
PATH = "../../../data/dogscats/"
sz=224

In [ ]:
# verify GPU
print(torch.cuda.is_available(), torch.backends.cudnn.enabled)

## Inspect data

In [ ]:
os.listdir(PATH)

In [ ]:
os.listdir(f'{PATH}valid')

In [ ]:
files = os.listdir(f'{PATH}valid/cats')[:5]
files

In [ ]:
img = plt.imread(f'{PATH}valid/cats/{files[0]}')
plt.imshow(img);

In [ ]:
img.shape

In [ ]:
img[:4,:4]

## Model

In [ ]:
arch=resnet34
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 2)

In [ ]:
# This is the label for a val data
data.val_y

In [ ]:
# from here we know that 'cats' is label 0 and 'dogs' is label 1.
data.classes

In [ ]:
# this gives prediction for validation set. Predictions are in log scale
log_preds = learn.predict()
log_preds.shape

In [ ]:
log_preds[:10]

In [ ]:
preds = np.argmax(log_preds, axis=1)  # from log probabilities to 0 or 1
probs = np.exp(log_preds[:,1])        # pr(dog)

### Choose a learning rate

In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute=True)


In [ ]:

lrf=learn.lr_find()

In [ ]:

learn.sched.plot_lr()

In [ ]:
learn.sched.plot()


### Data augmentation

In [ ]:
tfms = tfms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.1)


In [ ]:
def get_augs():
    data = ImageClassifierData.from_paths(PATH, bs=2, tfms=tfms, num_workers=1)
    x,_ = next(iter(data.aug_dl))
    return data.trn_ds.denorm(x)[1]

In [ ]:
ims = np.stack([get_augs() for i in range(6)])

In [ ]:

plots(ims, rows=2)

In [ ]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [ ]:
learn.fit(1e-2, 1)

In [ ]:
learn.precompute=False

In [ ]:
learn.fit(1e-2, 3, cycle_len=1)

In [ ]:

learn.sched.plot_lr()

In [ ]:
learn.save('224_lastlayer')

In [ ]:

learn.load('224_lastlayer')

In [ ]:
learn.unfreeze()


In [ ]:
lr=np.array([1e-4,1e-3,1e-2])


In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)


In [ ]:
learn.sched.plot_lr()


In [ ]:
learn.save('224_all')


In [ ]:
learn.load('224_all')


## Results

In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

In [ ]:

accuracy_np(probs, y)

In [ ]:

preds = np.argmax(probs, axis=1)
probs = probs[:,1]

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

In [ ]:
plot_confusion_matrix(cm, data.classes)

In [ ]:
def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8)) if len(imgs)>0 else print('Not Found.')

In [ ]:
plot_val_with_title(most_by_correct(0, False), "Most incorrect cats")


In [ ]:
plot_val_with_title(most_by_correct(1, False), "Most incorrect dogs")
